In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import pandas as pd
import time
import math
import sys
import random
from ast import literal_eval

from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

%matplotlib inline

# Data Initialization

In [3]:
classes = ["adult_males", "subadult_males", "adult_females", "juveniles", "pups"]

train_path = '/Users/YINAN/Local/Sea-lions/Data/Train/'
train_dotted_path = '/Users/YINAN/Local/Sea-lions/Data/TrainDotted/'

bad_images = [3,7,9,21,30,34,71,81,89,97,151,184,215,234,242,268,290,311,331,344,380,384,406,421,469,475,490,499,507,
              530,531,605,607,614,621,638,644,687,712,721]

file_names = [str(x) + '.jpg' for x in range(0,750) if x not in bad_images]
coordinates_df = pd.DataFrame(index=file_names, columns=classes)
# read all the coordinates 
coordinates_df = pd.read_csv("./coordinates.csv", index_col='index', converters={"adult_males": literal_eval,\
"subadult_males": literal_eval, "adult_females": literal_eval, "juveniles": literal_eval, "pups": literal_eval, "total": literal_eval})

In [4]:
coordinates_df.head()

,adult_males,subadult_males,adult_females,juveniles,pups,total
index,,,,,,
0.jpg,"[(2353, 3726), (3638, 3539), (4720, 3476), (41...","[(1437, 3178), (4017, 3094), (5135, 2649), (63...","[(1953, 3730), (2882, 3728), (3443, 3725), (26...","[(3295, 3515), (1009, 3489), (3941, 3470), (10...","[(3143, 3732), (3486, 3726), (2694, 3711), (19...","[(3143, 3732), (1953, 3730), (2882, 3728), (34..."
1.jpg,"[(3012, 2880), (2233, 871)]","[(4655, 3056), (4574, 2992), (4609, 2959), (39...",[],"[(3619, 3609), (4781, 3414), (4437, 2933), (43...",[],"[(3619, 3609), (4781, 3414), (4655, 3056), (45..."
2.jpg,"[(2272, 1816), (2084, 1773)]",[],"[(2306, 1882), (2226, 1877), (2272, 1858), (23...","[(2342, 1887), (2332, 1880), (2037, 1860), (22...",[],"[(2342, 1887), (2306, 1882), (2332, 1880), (22..."
4.jpg,"[(2458, 3367), (60, 2535), (3088, 2503), (3347...","[(1693, 3505), (2143, 2954), (2966, 2923), (91...","[(675, 3479), (1093, 3465)]",[],[],"[(1693, 3505), (675, 3479), (1093, 3465), (245..."
5.jpg,"[(3952, 2911), (3053, 2483), (2996, 2154), (30...","[(3488, 3218), (3570, 1342), (3366, 770), (340...","[(3301, 1796), (3170, 1790), (3246, 1711), (30...","[(3214, 1769), (3188, 1705), (3147, 1684), (33...","[(3212, 1811), (3301, 1770), (3299, 1732), (32...","[(3488, 3218), (3952, 2911), (3053, 2483), (29..."


In [5]:
def sliding_window(image):
    patches = []
    for row in range(image.shape[0]//96):
        for col in range(image.shape[1]//96):
            patch_rgb = cv2.cvtColor(image[row*96:row*96+96, col*96:col*96+96], cv2.COLOR_BGR2RGB)
            patches.append(patch_rgb)
    return patches

def extract_patches_with_sealions(coordinates_df, num_patches):
    patches = {
        "adult_males": [], 
        "subadult_males": [], 
        "adult_females": [], 
        "juveniles": [], 
        "pups": []
    }
    classes = ["adult_males", "subadult_males", "adult_females", "juveniles", "pups"]
    coordinates_df = coordinates_df.sample(frac=1)
    for filename in coordinates_df.index:
        if len(classes) == 0:
            return patches
        image = cv2.imread(train_path + filename)
        for sealion_class in classes:
            for coordinates in coordinates_df.loc[filename][sealion_class]:
                thumb = image[coordinates[1]-48:coordinates[1]+48,coordinates[0]-48:coordinates[0]+48,:]
                if np.shape(thumb) == (96, 96, 3):
                    patches[sealion_class].append(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB))
                    if len(patches[sealion_class]) == num_patches[sealion_class]:
                        print("%s patches completes, number = %d"%(sealion_class, len(patches[sealion_class])))
                        classes.remove(sealion_class)
                        break

def extract_patches_without_sealions(coordinates_df, num_patches):
    patches = []
    coordinates_df = coordinates_df.sample(frac=1)
    for filename in coordinates_df.index:
        sealion_coordinates_list = coordinates_df.loc[filename].total
        image = cv2.imread(train_path + filename)
        for row in range(image.shape[0]//96):
            for col in range(image.shape[1]//96):
                center = (row*96+48, col*96+48)
                flag = True
                for thumb in sealion_coordinates_list:
                    if math.sqrt((center[0] - thumb[1])**2 + (center[1] - thumb[0])**2) < math.sqrt(2)*96:
                        flag = False
                        break
                if flag:
                    patch_rgb = cv2.cvtColor(image[row*96:row*96+96, col*96:col*96+96], cv2.COLOR_BGR2RGB)
                    patches.append(patch_rgb)
                    print("\r%d patches completes, with total %d"%(len(patches), num_patches), end='')
                    sys.stdout.flush()
                    if len(patches) == num_patches:
                        return patches

def generate_dataset(coordinate_df, num_adult_males, num_subadult_males, num_adult_females, num_juveniles, num_pups, num_background):
    sealion_patches_num = {
        'adult_males': num_adult_males,
        'subadult_males': num_subadult_males,
        'adult_females': num_adult_females,
        'juveniles': num_juveniles,
        'pups':num_pups
    }
    sealion_patches = extract_patches_with_sealions(coordinates_df, sealion_patches_num)
    background_patches = extract_patches_without_sealions(coordinates_df, num_background)
    x = []
    y = []
    for key, val in sealion_patches.items():
        x += val
        y += [key]*len(val)
    for val in background_patches:
        x.append(val)
        y.append('background')
    x = np.array(x)
    y = np.array(y)
    return x, y

In [5]:
x, y = generate_dataset(coordinates_df, num_adult_males=1000, num_subadult_males=1000, num_adult_females=1000,\
                    num_juveniles=1000,num_pups=1000, num_background=20000)

adult_females patches completes, number = 1000
juveniles patches completes, number = 1000
pups patches completes, number = 1000
adult_males patches completes, number = 1000
subadult_males patches completes, number = 1000
20000 patches completes, with total 20000

# Keras 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Lambda, Cropping2D
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.utils import np_utils

Using TensorFlow backend.
/Users/YINAN/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)
x_train = x_train/255
x_test = x_test/255
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_train = np_utils.to_categorical(y_train, 10)
y_test = encoder.transform(y_test)
y_test= np_utils.to_categorical(y_test, 10)

In [7]:
INPUT_SHAPE = (96,96,3)

model = Sequential()
# First layer
model.add(Convolution2D(8, (5, 5), activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Second layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Third layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# Fourth layer
model.add(Convolution2D(10, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())

# model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 92, 92, 8)         608       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 46, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 5)         365       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 5)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 5)         230       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 5)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 10)          460       
__________

In [11]:
model.fit(x_train, y_train, 
          batch_size=50, epochs=50, verbose=1)

Epoch 1/50
17500/17500 [==============================] - 200s 11ms/step - loss: 0.7977 - acc: 0.8009
Epoch 2/50
17500/17500 [==============================] - 165s 9ms/step - loss: 0.4723 - acc: 0.8358
Epoch 3/50
17500/17500 [==============================] - 2466s 141ms/step - loss: 0.4088 - acc: 0.8596
Epoch 4/50
17500/17500 [==============================] - 166s 9ms/step - loss: 0.3943 - acc: 0.8624
Epoch 5/50
17500/17500 [==============================] - 138s 8ms/step - loss: 0.3834 - acc: 0.8671
Epoch 6/50
17500/17500 [==============================] - 139s 8ms/step - loss: 0.3678 - acc: 0.8724
Epoch 7/50
17500/17500 [==============================] - 127s 7ms/step - loss: 0.3652 - acc: 0.8746
Epoch 8/50
17500/17500 [==============================] - 127s 7ms/step - loss: 0.3575 - acc: 0.8755
Epoch 9/50
17500/17500 [==============================] - 125s 7ms/step - loss: 0.3465 - acc: 0.8802
Epoch 10/50
17500/17500 [==============================] - 115s 7ms/step - loss: 0.3432

In [12]:
model.evaluate(x_test, y_test)

7500/7500 [==============================] - 47s 6ms/step


[0.26819540227254229, 0.9032]

In [13]:
# save the model
model.save('./cnn_saved_model.h5')

In [14]:
from keras.models import load_model
old = load_model('./cnn_saved_model.h5')

In [15]:
old

In [16]:
old.evaluate(x_train, y_train)

17500/17500 [==============================] - 95s 5ms/step


[0.23468859199455805, 0.91548571429933823]

# generate new test data to check the model

In [17]:
new_x, new_y = generate_dataset(coordinates_df, num_adult_males=123, num_subadult_males=345, num_adult_females=111,\
                    num_juveniles=451,num_pups=500, num_background=2000)

adult_females patches completes, number = 111
juveniles patches completes, number = 451
adult_males patches completes, number = 123
pups patches completes, number = 500
subadult_males patches completes, number = 345
2000 patches completes, with total 2000

In [18]:
new_x = new_x/255
encoder = LabelEncoder()
encoder.fit(new_y)
new_y = encoder.transform(new_y)
new_y = np_utils.to_categorical(new_y, 10)

In [1]:
model.evaluate

NameError: name 'model' is not defined